In [39]:
%esptool --port COM4 erase

Executing:
  esptool --port COM4 erase_flash

esptool.py v2.8
Serial port COM4
Connecting........_
[Press the PRG button now if required]
Detecting chip type... ESP32
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: cc:50:e3:80:1f:20
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 3.0s
Hard resetting via RTS pin...


In [40]:
%esptool --port com4 esp32 ../scripts/esp32-idf3-20191109-v1.11-571-g7e374d231.bin

Executing:
  esptool --port com4 --chip esp32 write_flash -z 0x1000 ../scripts/esp32-idf3-20191109-v1.11-571-g7e374d231.bin

esptool.py v2.8
Serial port com4
Connecting....
[Press the PRG button now if required]
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: cc:50:e3:80:1f:20
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1240160 bytes to 783102...
Wrote 1240160 bytes (783102 compressed) at 0x00001000 in 69.7 seconds (effective 142.4 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [55]:
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 
pu up.
ap config here!
('ap_config:', 'informatics\n0953313123')
line to:  ('informatics', '0953313123')
>, old:<1,0>, ap:<255,255>, sta:<7,1>, prof:1
> auth (b0)
> assoc (0)
> run (10)

connected!  -->  ('192.168.43.150', '255.255.255.0', '192.168.43.254', '120.118.253.1')
Traceback (most recent call last):
  File "main.py", line 54, in <module>
RuntimeError: Wifi Unknown Error 0x0005
MicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [56]:
import network
import time

In [57]:
sta = network.WLAN(network.STA_IF)
print(sta.active(True))
SSID='informatics'
KEY='0953313123'
print(sta.active())
sta.connect(SSID,KEY)

True
True


In [58]:
import network
import ubinascii
mac = ubinascii.hexlify(sta.config('mac'),':').decode()
print(mac)
print(sta.ifconfig())

cc:50:e3:80:1f:20
('192.168.43.150', '255.255.255.0', '192.168.43.254', '120.118.253.1')


In [59]:
print(sta.isconnected())
for i in range(20):
    time.sleep(0.5)
    if sta.isconnected():
        break
print(sta.ifconfig())

True
('192.168.43.150', '255.255.255.0', '192.168.43.254', '120.118.253.1')


In [46]:
%sendtofile ap.txt
informatics
0953313123

Sent 2 lines (22 bytes) to ap.txt.


In [62]:
%sendtofile main.py
# -*- coding: utf-8 -*-

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = 'pypypypy'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04


# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)


# th_sensor
th_sensor = dht.DHT11(Pin(5))


#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))

    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    th_sensor_read = request.find('GET /?th_sensor=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)
   

    my_t = None
    my_h = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()    
        my_t = th_sensor.temperature()  
        my_h = th_sensor.humidity()
        print('T=', th_sensor.temperature())
        print('H=', th_sensor.humidity())

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?', 'T={}{}C'.format(my_t,'\u00b0'))
        else:
            html = html.replace('T=?', '')
        if my_h:
            html = html.replace('H=?', 'H={}{}'.format(my_h,'%'))
        else:
            html = html.replace('H=?', '')
        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()

Sent 120 lines (2632 bytes) to main.py.


In [61]:
%sendtofile webtool.html
<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D4):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D1):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?unit
</form>

</html>

Sent 39 lines (789 bytes) to webtool.html.


In [53]:
%sendtofile rdht.py
from machine import Pin
import dht     
import time

p5=Pin(5, Pin.IN)
d=dht.DHT11(p5)        #建立 DHT11 物件

d.measure()                  #重新測量溫溼度
t=d.temperature()        #讀取攝氏溫度
h=d.humidity()             #讀取相對溼度
print("T={},H={}".format(t,h))

Sent 11 lines (244 bytes) to rdht.py.


In [63]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x1f6cdc83088, open=True>(port='COM4', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
